This Python Jupyter Notebook creates the SQL Server tables for the UK General Election model, in accordance with the entity-relationship diagram version 2.1:

https://github.com/danielunderwood2292/Public/blob/main/Elections/0_Elections_Entity_Relationship_Diagram.svg

In [1]:
#Import required packages
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib

In [2]:
#Connect to database 'UK_General_Election' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=DANZPOOTA;DATABASE=UK_General_Election;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [3]:
#Procedure to delete the tables in the database if they already exist
DeleteTables = """DROP TABLE IF EXISTS ElectionPredictionOverall
DROP TABLE IF EXISTS ElectionPredictionConstituencies
DROP TABLE IF EXISTS ElectionPredictionCandidates
DROP TABLE IF EXISTS ElectionPredictionPollsUsed
DROP TABLE IF EXISTS ElectionPredictionMeta
DROP TABLE IF EXISTS PollAnalysisConstituencies
DROP TABLE IF EXISTS PollAnalysisRegions
DROP TABLE IF EXISTS PollAnalysisMeta
DROP TABLE IF EXISTS PollDetails
DROP TABLE IF EXISTS PollMeta
DROP TABLE IF EXISTS Pollsters
DROP TABLE IF EXISTS RegionConstituencies
DROP TABLE IF EXISTS RegionRegionTypes
DROP TABLE IF EXISTS Regions
DROP TABLE IF EXISTS RegionTypes
DROP TABLE IF EXISTS Candidates
DROP TABLE IF EXISTS Constituencies
DROP TABLE IF EXISTS Parties"""

In [4]:
engine.execute(DeleteTables)

In [5]:
CreateRegionTypes = """Create Table RegionTypes
    (RegionType VARCHAR(25) PRIMARY KEY NOT NULL,
    RegionTypeRank INT NOT NULL)"""

In [6]:
engine.execute(CreateRegionTypes)

In [7]:
CreatePollsters = """Create Table Pollsters
    (
    PollsterName VARCHAR(100) PRIMARY KEY NOT NULL,
    DefaultRegionType VARCHAR(25) FOREIGN KEY REFERENCES RegionTypes(RegionType)
    )"""

In [8]:
engine.execute(CreatePollsters)

In [9]:
CreateRegions = """Create Table Regions
    (RegionName VARCHAR(50) PRIMARY KEY NOT NULL)"""

In [10]:
engine.execute(CreateRegions)

In [11]:
CreateRegionRegionTypes = """Create Table RegionRegionTypes
    (
    RegionXTypesID AS RegionType + RegionName PERSISTED PRIMARY KEY NOT NULL,
    RegionName VARCHAR(50) FOREIGN KEY REFERENCES Regions(RegionName) NOT NULL,
    RegionType VARCHAR(25) FOREIGN KEY REFERENCES RegionTypes(RegionType) NOT NULL
    )"""

In [12]:
engine.execute(CreateRegionRegionTypes)

In [13]:
CreateParties = """Create Table Parties
    (
    PartyAbbreviation VARCHAR(8) PRIMARY KEY NOT NULL,
    PartyFullName VARCHAR(40)
    )"""

In [14]:
engine.execute(CreateParties)

In [15]:
CreateConstituencies = """Create Table Constituencies
    (
    ONSID CHAR(9) NOT NULL,
    PANO INT NOT NULL,
    ConstituencyName VARCHAR(50) PRIMARY KEY NOT NULL,
    ClosestOldConstituency VARCHAR(50) NOT NULL,
    PAName VARCHAR(50) NOT NULL,
    Nation VARCHAR(20) NOT NULL,
    ConstituencyType VARCHAR(10) NOT NULL,
    ITL1Region VARCHAR(25) NOT NULL,
    FirstParty VARCHAR(8) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    SecondParty VARCHAR(8) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    Electorate INT NOT NULL,
    TotalVotes INT NOT NULL,
    MajorityVotes INT NOT NULL,
    MajorityShare DECIMAL(9,8) NOT NULL
    )"""

In [16]:
engine.execute(CreateConstituencies)

In [17]:
CreateRegionConstituencies = """Create Table RegionConstituencies
    (
    RegionConsID AS RegionName + ConstituencyName PERSISTED PRIMARY KEY NOT NULL,
    ConstituencyName VARCHAR(50) FOREIGN KEY REFERENCES Constituencies(ConstituencyName) NOT NULL,
    RegionName VARCHAR(50) FOREIGN KEY REFERENCES Regions(RegionName) NOT NULL 
    )"""

In [18]:
engine.execute(CreateRegionConstituencies)

In [19]:
# Example constraint: https://stackoverflow.com/questions/7844460/foreign-key-to-multiple-tables
CreatePollMeta = """Create Table PollMeta
    (
    PollID AS CONVERT(CHAR(8),PollDate,112) + Pollster + PollType + PollScope PERSISTED PRIMARY KEY,
    Pollster VARCHAR(100) FOREIGN KEY REFERENCES Pollsters(PollsterName) NOT NULL,
    PollType VARCHAR(25) FOREIGN KEY REFERENCES RegionTypes(RegionType) NOT NULL,
    PollDate DATE NOT NULL,
    PollScope VARCHAR(50) NOT NULL,
    PollScopeAll BIT NOT NULL,
    PollScopeRegion VARCHAR(50) FOREIGN KEY REFERENCES Regions(RegionName),
    PollScopeConst VARCHAR(50) FOREIGN KEY REFERENCES Constituencies(ConstituencyName),
    SampleSize INT,
    CONSTRAINT CheckPollScope CHECK(
        PollScopeAll +
        CASE WHEN PollScopeRegion IS NULL THEN 0 ELSE 1 END +
        CASE WHEN PollScopeConst IS NULL THEN 0 ELSE 1 END
        = 1),
    PollLink varchar(255)
    )"""

In [20]:
engine.execute(CreatePollMeta)

In [21]:
CreatePollDetails = """Create Table PollDetails
    (
    PollDetailsID AS PollID + RegionName + Party PERSISTED PRIMARY KEY,
    PollID VARCHAR(183) FOREIGN KEY REFERENCES PollMeta(PollID) NOT NULL,
    RegionName VARCHAR(50) FOREIGN KEY REFERENCES Regions(RegionName) NOT NULL,
    Party VARCHAR(8) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    VoteShare DECIMAL(9,8) NOT NULL
    )"""

In [22]:
engine.execute(CreatePollDetails)

In [23]:
CreateCandidates = """Create Table Candidates
    (
    CandidateID AS Constituency + Party PERSISTED PRIMARY KEY,
    Constituency VARCHAR(50) FOREIGN KEY REFERENCES Constituencies(ConstituencyName) NOT NULL,
    Party VARCHAR(8) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    SittingMP BIT,
    FormerMP BIT,
    FirstName VARCHAR(25),
    Surname VARCHAR(25),
    Gender VARCHAR(10),
    PreviousVotes INT NOT NULL,
    PreviousShare DECIMAL(9,8) NOT NULL,
    PreviousStanding INT NOT NULL,
    CurrentStanding INT NOT NULL
    )"""

In [24]:
engine.execute(CreateCandidates)

In [25]:
CreatePollAnalysisMeta = """Create Table PollAnalysisMeta
    (
    PollAnalysisID AS CONVERT(CHAR(8),PollAnalysisDate,112) + PollID + PollAnalysisAlgorithm PERSISTED PRIMARY KEY,
    PollAnalysisDate DATE NOT NULL,
    PollID VARCHAR(183) FOREIGN KEY REFERENCES PollMeta(PollID) NOT NULL,
    PollAnalysisAlgorithm VARCHAR(50) NOT NULL
    )"""

In [26]:
engine.execute(CreatePollAnalysisMeta)

In [27]:
CreatePollAnalysisRegions = """Create Table PollAnalysisRegions
    (
    PollAnalysisRegionID AS PollDetailsID + PollAnalysisID PERSISTED PRIMARY KEY,
    PollDetailsID VARCHAR(241) FOREIGN KEY REFERENCES PollDetails(PollDetailsID) NOT NULL,
    PollAnalysisID VARCHAR(241) FOREIGN KEY REFERENCES PollAnalysisMeta(PollAnalysisID) NOT NULL,
    Swing DECIMAL(9,8) NOT NULL
    )"""

In [28]:
engine.execute(CreatePollAnalysisRegions)

In [29]:
CreatePollAnalysisConstituencies = """Create Table PollAnalysisConstituencies
    (
    PollAnalysisRegionID VARCHAR(482) FOREIGN KEY REFERENCES PollAnalysisRegions(PollAnalysisRegionID) NOT NULL,
    CandidateID VARCHAR(58) FOREIGN KEY REFERENCES Candidates(CandidateID) NOT NULL,
    PollAnalysisConstituenciesID AS PollAnalysisRegionID + CandidateID PERSISTED PRIMARY KEY,    
    VoteShare DECIMAL(9,8) NOT NULL
    )"""

In [30]:
engine.execute(CreatePollAnalysisConstituencies)

In [31]:
CreateElectionPredictionMeta = """Create Table ElectionPredictionMeta
    (
    ElectionPredictionID AS CONVERT(CHAR(8),ElectionAnalysisDate,112)  + CONVERT(CHAR(8),ElectionPredictionDate,112) + ElectionAnalysisAlgorithm PERSISTED PRIMARY KEY,
    ElectionAnalysisDate Date NOT NULL,
    ElectionPredictionDate Date NOT NULL,
    ElectionAnalysisAlgorithm VARCHAR(50) NOT NULL
    )"""

In [32]:
engine.execute(CreateElectionPredictionMeta)

In [33]:
CreateElectionPredictionPollsUsed = """Create Table ElectionPredictionPollsUsed
    (
    EPPUID AS PollID + ElectionPredictionID PERSISTED PRIMARY KEY,
    PollID VARCHAR(183) FOREIGN KEY REFERENCES PollMeta(PollID) NOT NULL,
    ElectionPredictionID VARCHAR(66) FOREIGN KEY REFERENCES ElectionPredictionMeta(ElectionPredictionID) NOT NULL
    )"""

In [34]:
engine.execute(CreateElectionPredictionPollsUsed)

In [35]:
CreateElectionPredictionCandidates = """Create Table ElectionPredictionCandidates
    (
    ElectionPredictionCandID AS ElectionPredictionID + CandidateID PERSISTED PRIMARY KEY,
    ElectionPredictionID VARCHAR(66) FOREIGN KEY REFERENCES ElectionPredictionMeta(ElectionPredictionID) NOT NULL,
    CandidateID VARCHAR(58) FOREIGN KEY REFERENCES Candidates(CandidateID) NOT NULL,
    VoteShare DECIMAL(9,8) NOT NULL
    )"""

In [36]:
engine.execute(CreateElectionPredictionCandidates)

In [37]:
CreateElectionPredictionConstituencies = """Create Table ElectionPredictionConstituencies
    (
    ElectionPredictionConsID AS ElectionPredictionID + Constituency PERSISTED PRIMARY KEY,
    ElectionPredictionID VARCHAR(66) FOREIGN KEY REFERENCES ElectionPredictionMeta(ElectionPredictionID) NOT NULL,
    Constituency VARCHAR(50) FOREIGN KEY REFERENCES Constituencies(ConstituencyName) NOT NULL,
    WinningParty VARCHAR(8) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    SecondParty VARCHAR(8) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    VoteShare DECIMAL(9,8) NOT NULL,
    Majority DECIMAL(9,8) NOT NULL,
    PreviousWinner VARCHAR(8) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    GAIN VARCHAR(13) NOT NULL,
    LOSS VARCHAR(13) NOT NULL,
    Swing DECIMAL(9,8) NOT NULL
    )"""

In [38]:
engine.execute(CreateElectionPredictionConstituencies)

In [39]:
CreateElectionPredictionOverall = """Create Table ElectionPredictionOverall
    (
    ElectionPredictionOverID AS ElectionPredictionID + Party PERSISTED PRIMARY KEY,
    ElectionPredictionID VARCHAR(66) FOREIGN KEY REFERENCES ElectionPredictionMeta(ElectionPredictionID) NOT NULL,
    Party VARCHAR(8) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    VoteShare DECIMAL(9,8) NOT NULL,
    Constituencies INT NOT NULL
    )"""

In [40]:
engine.execute(CreateElectionPredictionOverall)

In [41]:
#Get the table names currently in the database
insp = sqlalchemy.inspect(engine)
print(insp.get_table_names())

['Candidates', 'Constituencies', 'ElectionPredictionCandidates', 'ElectionPredictionConstituencies', 'ElectionPredictionMeta', 'ElectionPredictionOverall', 'ElectionPredictionPollsUsed', 'Parties', 'PollAnalysisConstituencies', 'PollAnalysisMeta', 'PollAnalysisRegions', 'PollDetails', 'PollMeta', 'Pollsters', 'RegionConstituencies', 'RegionRegionTypes', 'Regions', 'RegionTypes']


In [42]:
conn.close()